In [1]:
import numpy as np

In [2]:
datContentTrain = [i.strip().split() for i in open("./hw6_train.dat").readlines()]
for i in range(len(datContentTrain)):
    for j in range(len(datContentTrain[i])):
        datContentTrain[i][j] = float(datContentTrain[i][j])
data_train = np.array(datContentTrain)

datContentTest = [i.strip().split() for i in open("./hw6_test.dat").readlines()]
for i in range(len(datContentTest)):
    for j in range(len(datContentTest[i])):
        datContentTest[i][j] = float(datContentTest[i][j])
data_test = np.array(datContentTest)

In [3]:
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

In [147]:
class Adaboost:
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.weak_clfs = None
        self.u = None

    def initialize(self):
        u = np.repeat(1/self.X_train.shape[0], self.X_train.shape[0])
        return u

    def binary_to_diamond_arr(self, arr, diamond):
        """ 
        arr: binary array
        diamond: diamond value in adaboost
        Map (0, 1) to (n, 1/n) 
        """
        arr_ = ( (1-diamond**2) / diamond ) * arr + diamond
        return arr_

    def decision_stump(self):
        best_i = 0
        best_err = np.inf            
        best_threshold = 0
        best_decision = 0
        best_pred = None

        for i in range(self.X_train.shape[1]):
            #select feature i 
            selected_features = np.sort(self.X_train[:, i])
            thresholds = np.concatenate(
                [np.array([-np.inf]), 
                (selected_features[1:] + selected_features[:-1]) / 2]
            )

            #test all threshold in features i 
            for threshold in thresholds:
                for decision in [1, -1]:
                    #when feature > threshold, predict +1
                    if decision == 1: 
                        y_pred = self.binary_to_sign(
                            self.X_train[:, i] >= threshold
                            )
                    #when feature > threshold, predict -1
                    else: 
                        y_pred = self.binary_to_sign(
                            self.X_train[:, i] < threshold
                            )                
                            
                    weight_err = self.compute_weighted_error(y_pred)
                    if weight_err < best_err:
                        best_i = i
                        best_err = weight_err
                        best_threshold = threshold
                        best_decision = decision
                        best_pred = y_pred
        return best_i, best_threshold, best_decision, best_err, best_pred

    def train(self, max_iter = 500, verbose = False):
        weak_clfs = {
            "iter": [],
            "feat_idx": [],
            "thres": [],
            "decision": [],
            "alpha": [],
            "g_err_rate": [],
            "g_weight_err_rate": [],
            "G_err_rate": [],
            "pred": []
        }
        strong_clf = np.zeros(self.X_train.shape[0])
        self.u = self.initialize()
        for i in range(max_iter):
            feature_idx, threshold, decision, weight_err, pred = self.decision_stump()
            clf_result = (pred == self.y_train)
            err_rate = np.sum(1-clf_result) / self.X_train.shape[0]
            weight_err_rate = weight_err / np.sum(self.u)
            diamond = np.sqrt(
                (1-weight_err_rate) / weight_err_rate
            )
            diamond_arr = self.binary_to_diamond_arr(clf_result, diamond)
            self.u = self.u * diamond_arr
            alpha = np.log(diamond)
            strong_clf += alpha * pred
            strong_clf_pred = np.sign(strong_clf)
            strong_clf_err_rate = np.sum(
                (strong_clf_pred != self.y_train)
            ) / self.X_train.shape[0]

            #store records
            weak_clfs["iter"].append(i)
            weak_clfs["feat_idx"].append(feature_idx)
            weak_clfs["thres"].append(threshold)
            weak_clfs["decision"].append(decision)
            weak_clfs["alpha"].append(alpha)
            weak_clfs["g_err_rate"].append(err_rate)
            weak_clfs["g_weight_err_rate"].append(weight_err_rate)
            weak_clfs["G_err_rate"].append(strong_clf_err_rate)
            weak_clfs["pred"].append(pred)

            if verbose:
                print(
                    f"iter {i+1}: feat idx = {feature_idx}, thres = {round(threshold, 3)}",
                    f"s = {decision}, alpha = {round(alpha, 3)}, E_in(g) = {round(err_rate*100, 3)}%", 
                    f", E_in(G) = {round(strong_clf_err_rate*100, 3)}%"
                    )
        if verbose:
            print(f"Final error rate: {round(strong_clf_err_rate * 100, 4)}%")
        self.weak_clfs = weak_clfs
        return weak_clfs

    def predict(self, X_test, y_test, num_iter, isUniform = False, verbose = False):
        g_err_rate_record = []
        G_err_rate_record = []
        strong_clf = np.zeros(self.X_train.shape[0])
        for idx in range(num_iter):
            feature_idx = self.weak_clfs["feat_idx"][idx]
            if self.weak_clfs["decision"][idx] == 1:
                weak_pred = self.binary_to_sign(
                    X_test[:, feature_idx] >= self.weak_clfs["thres"][idx]
                )
            else:
                weak_pred = self.binary_to_sign(
                    X_test[:, feature_idx] < self.weak_clfs["thres"][idx]
                )
            if isUniform:
                strong_clf += weak_pred
            else:
                strong_clf += self.weak_clfs["alpha"][idx] * weak_pred
            strong_clf_pred = np.sign(strong_clf)

            g_err_rate = self.compute_error_rate(
                weak_pred, y_test
            )
            G_err_rate = self.compute_error_rate(
                strong_clf_pred, y_test
            )
            g_err_rate_record.append(g_err_rate)
            G_err_rate_record.append(G_err_rate)
            if verbose:
                print(f"Iter {idx}: E_in(g) = {round(g_err_rate, 4)}%, E_in(G) = {round(G_err_rate, 4)}%")
        return g_err_rate_record, G_err_rate_record

    def compute_error_rate(self, y_pred, y_true):
        err_rate = np.sum(
                (y_pred != y_true)
            ) / self.X_train.shape[0]
        return err_rate

    def compute_weighted_error(self, y_pred):
        return np.dot((y_pred != y_train), self.u)

    def binary_to_sign(self, arr):
        """
        Map (0, 1) to (-1, 1)
        """
        return arr * 2 - 1

# Problem 11

In [52]:
num_iter = 1
adaboost_clf = Adaboost(X_train, y_train)
clf_results = adaboost_clf.train(num_iter, verbose=True)

iter 1: feat idx=9, thres = 0.448 s = -1, alpha = 0.258, E_in(g) = 37.4% , E_in(G) = 37.4%
Final error rate: 37.4%


# Problem 12

In [148]:
num_iter = 500
adaboost_clf = Adaboost(X_train, y_train)
clf_result = adaboost_clf.train(num_iter, verbose=True)

iter 1: feat idx = 9, thres = 0.448 s = -1, alpha = 0.258, E_in(g) = 37.4% , E_in(G) = 37.4%
iter 2: feat idx = 0, thres = 2.679 s = 1, alpha = 0.212, E_in(g) = 42.8% , E_in(G) = 37.4%
iter 3: feat idx = 0, thres = -1.514 s = -1, alpha = 0.205, E_in(g) = 44.1% , E_in(G) = 31.8%
iter 4: feat idx = 5, thres = -1.247 s = 1, alpha = 0.283, E_in(g) = 39.6% , E_in(G) = 34.5%
iter 5: feat idx = 0, thres = 3.826 s = 1, alpha = 0.177, E_in(g) = 45.9% , E_in(G) = 30.4%
iter 6: feat idx = 8, thres = -0.587 s = 1, alpha = 0.188, E_in(g) = 41.4% , E_in(G) = 30.8%
iter 7: feat idx = 0, thres = -2.253 s = -1, alpha = 0.179, E_in(g) = 44.0% , E_in(G) = 30.2%
iter 8: feat idx = 7, thres = -0.676 s = 1, alpha = 0.201, E_in(g) = 42.1% , E_in(G) = 29.7%
iter 9: feat idx = 5, thres = 2.947 s = 1, alpha = 0.13, E_in(g) = 49.9% , E_in(G) = 29.1%
iter 10: feat idx = 8, thres = 0.518 s = -1, alpha = 0.18, E_in(g) = 43.8% , E_in(G) = 25.6%
iter 11: feat idx = 0, thres = 1.447 s = 1, alpha = 0.127, E_in(g) = 44.

In [99]:
max_e_in = max(clf_result["g_err_rate"])
print(f"Maximum E_in(g) in 500 iteration is {max_e_in}")

Maximum E_in(g) in 500 iteration is 0.591


# Problem 13

In [100]:
iter_list = [60, 160, 260, 360, 460]
for iter_ in iter_list:
    G_in = clf_result["G_err_rate"][:iter_-1]
    print(f"Choice {iter_}: E_in(G) = {round(min(G_in)*100, 4)}%")

Choice 60: E_in(G) = 10.7%
Choice 160: E_in(G) = 7.0%
Choice 260: E_in(G) = 5.6%
Choice 360: E_in(G) = 5.0%
Choice 460: E_in(G) = 4.7%


In [101]:
threshold = 0.05
for idx, E_in_G in enumerate(clf_result["G_err_rate"]):
    if E_in_G <= 0.05:
        print(f"The first iteration that E_in(G) <= {threshold} is iteration {idx+1}.")
        break

The first iteration that E_in(G) <= 0.05 is iteration 355.


# Problem 14

In [150]:
num_iter = 1
_, _ = adaboost_clf.predict(X_test, y_test, num_iter, isUniform=False, verbose = True)

Iter 0: E_in(g) = 0.455%, E_in(G) = 0.455%


# Problem 15

In [149]:
num_iter = 500
_, G_err_test_results = adaboost_clf.predict(X_test, y_test, num_iter, isUniform=True, verbose = False)
print(f"E_out(G_uniform) is {G_err_test_results[-1]}")

E_out(G_uniform) is 0.229


# Problem 16

In [152]:
num_iter = 500
_, G_err_test_results = adaboost_clf.predict(X_test, y_test, num_iter, isUniform=False, verbose = False)
print(f"E_out(G_500) is {G_err_test_results[-1]}")

E_out(G_500) is 0.188
